In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn
import theano.tensor as tt
import warnings
from IPython.core.pylabtools import figsize
import seaborn as sns
from sklearn.metrics import (roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, 
                             precision_recall_curve) 
from sklearn.metrics import confusion_matrix

In [ ]:
df=pd.read_csv('preprocessed.csv',sep = ',')
#def format_data(df):
#    print(df.info())
#    # Targets are final grade of student
#    labels = df['units_sold']
#    normalized_df=( labels-labels.mean())/ labels.std()
#    return normalized_df
#df=format_data(df)

In [ ]:
df.head()

,Unnamed: 0,price,retail_price,units_sold,uses_ad_boosts,rating,rating_five_count,rating_four_count,rating_three_count,rating_two_count,...,size__XXL,size__XXS,size__XXXS,size__XXXXL,size__XXXXXL,country__CN,country__GB,country__SG,country__US,country__VE
0,0,16.00,14,100,0,3.76,0.481481,0.148148,0.185185,0.018519,...,0,0,0,0,0,1,0,0,0,0
1,1,8.00,22,20000,1,3.45,0.369845,0.167400,0.182233,0.104971,...,0,0,0,0,0,1,0,0,0,0
2,2,8.00,43,100,0,3.57,0.357143,0.285714,0.142857,0.000000,...,0,0,0,0,0,1,0,0,0,0
3,3,8.00,8,5000,1,4.03,0.509499,0.205527,0.150259,0.072539,...,0,0,0,0,0,1,0,0,0,0
4,4,2.72,3,100,1,3.10,0.300000,0.200000,0.100000,0.100000,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
#The first model with priors having uniform distributions
lower=-10**6
higher=10**6
with pm.Model() as first_model:
    #priors on parameters
    beta_0=pm.Uniform('beta_0', lower=lower, upper= higher)
    beta_rating=pm.Uniform('beta_rating', lower, higher)
    beta_merch_pop_proc=pm.Uniform('beta_merch_pop_proc', lower, higher)
    beta_price=pm.Uniform('beta_price', lower, higher)
    beta_retail_price=pm.Uniform('beta_retail_price', lower, higher)
    #beta_humid_ratio=pm.Uniform('beta_humid_ration', lower, higher)
    
    #the probability of belonging to class 1
    p = pm.Deterministic('p', pm.math.sigmoid(beta_0+beta_rating*df['rating']+
                               beta_merch_pop_proc*df['merch_pop_proc']+
                               beta_price*df['price']+
                               beta_retail_price*df['retail_price']))
with first_model:
    #fit the data 
    observed=pm.Bernoulli("units_sold", p, observed=df['units_sold'])
    start=pm.find_MAP()
    step=pm.Metropolis()
    
    #samples from posterior distribution 
    trace=pm.sample(25000, step=step, start=start)
    burned_trace=trace[15000:]

logp = -inf, ||grad|| = 0: 100%|████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 668.47it/s]
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_retail_price]
>Metropolis: [beta_price]
>Metropolis: [beta_merch_pop_proc]
>Metropolis: [beta_rating]
>Metropolis: [beta_0]


In [ ]:
pm.traceplot(burned_trace)
plt.show()

ValueError: Can not extract posterior from <MultiTrace: 1 chains, 0 iterations, 11 variables>! See c:\users\chloetang\anaconda3\envs\bayesian test\lib\site-packages\arviz\data\converters.py for other conversion utilities.